In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.table import Table
import seaborn as sns
import numpy as np
from math import sqrt, atan, degrees

mlb_data = pd.read_csv('skubal-2024.csv')

In [25]:

# Filtrar lanzamientos que son strikes
strikes_data = mlb_data[mlb_data['description'].isin(['called_strike', 'swinging_strike', 'foul_tip'])]

# Cambiar la descripción a 'strikeout' si la cuenta tiene dos strikes
strikes_data['description'] = strikes_data.apply(
    lambda row: 'strikeout' if row['count'].count('-') == 1 and row['count'].split('-')[1] == '2' else row['description'],
    axis=1
)

# Función para obtener las dos últimas secuencias de lanzamientos antes de un strike
def get_pitch_sequences(strikes_df):
    sequences = []
    
    # Agrupar por cada bateador y juego
    grouped = strikes_df.groupby(['batter', 'game_date'])
    
    for (batter, game_date), group in grouped:
        # Ordenar los lanzamientos por la cuenta
        group = group.sort_values('count')
        
        for index, row in group.iterrows():
            # Obtener la cuenta actual
            current_count = row['count']
            
            # Filtrar lanzamientos previos a este strike
            prior_pitches = group[group['count'] < current_count].tail(2)  # Obtiene los 2 últimos lanzamientos
            
            if len(prior_pitches) == 2:  # Asegurarse de tener exactamente 2 lanzamientos
                sequence = prior_pitches['pitch_name'].tolist()
                strike = row['pitch_name']  # En este caso, se almacenará 'strikeout' si corresponde
                sequences.append((batter, game_date, current_count, sequence, strike))
            elif len(prior_pitches) == 1:  # Si solo hay 1 pitcheo anterior
                sequence = prior_pitches['pitch_name'].tolist()
                strike = row['pitch_name']
                sequences.append((batter, game_date, current_count, sequence, strike))

    return sequences

# Obtener las secuencias de pitcheo que llevaron a un strike
pitch_sequences = get_pitch_sequences(strikes_data)

# Convertir a DataFrame para análisis
sequences_df = pd.DataFrame(pitch_sequences, columns=['Batter ID', 'Game Date', 'Count', 'Last 2 Pitches', 'Strike'])
print(sequences_df)
#sequences_df.to_csv('1-2024.csv', index=False)


C:\Users\luism\AppData\Local\Temp\ipykernel_17668\1190299128.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strikes_data['description'] = strikes_data.apply(


     Batter ID   Game Date Count             Last 2 Pitches     Strike
0       455117  2024-03-28   0-2        [Slider, Four-Seam]  Four-Seam
1       457705  2024-05-29   0-1        [Slider, Four-Seam]     Slider
2       457705  2024-05-29   0-2        [Four-Seam, Slider]  Four-Seam
3       457705  2024-05-29   2-2        [Slider, Four-Seam]     Sinker
4       457759  2024-08-07   1-1                [Four-Seam]     Sinker
..         ...         ...   ...                        ...        ...
560     694671  2024-06-03   0-1    [Slider, Knuckle Curve]   Changeup
561     694671  2024-06-03   3-0  [Knuckle Curve, Changeup]  Four-Seam
562     694671  2024-06-03   3-2      [Changeup, Four-Seam]   Changeup
563     696100  2024-09-12   0-2                [Four-Seam]  Four-Seam
564     805779  2024-09-06   2-2                   [Sinker]   Changeup

[565 rows x 5 columns]
